In [1]:
!python -V

Python 3.9.12


In [2]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

# Q1. Downloading the data

In [3]:
df_jan = pd.read_parquet("../data/fhv_tripdata_2021-01.parquet")

In [4]:
df_jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


Read the data for January. How many records are there?

In [5]:
df_jan.shape[0]

1154112

# Q2. Computing duration

In [6]:
df_jan['duration'] = (df_jan.dropOff_datetime - df_jan.pickup_datetime)
df_jan['duration'] = df_jan.duration.apply(lambda x: x.total_seconds()/60)

In [8]:
print(f"{df_jan[~((df_jan.duration >= 1) & (df_jan.duration <=60))].shape[0]} rows was dropped")
df_jan = df_jan[((df_jan.duration >= 1) & (df_jan.duration <=60))]

44286 rows was dropped


In [9]:
df_jan.PUlocationID.fillna("-1", inplace=True)
df_jan.DOlocationID.fillna("-1", inplace=True)

In [10]:
int(100*(df_jan.PUlocationID.value_counts()[0]/df_jan.shape[0]))

83

# Q4. One-hot encoding

In [11]:
categorical = ['PUlocationID', 'DOlocationID']
df_jan[categorical] = df_jan[categorical].astype(str)

In [12]:
train_dicts = df_jan[categorical].to_dict(orient='records')

In [13]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [14]:
X_train.get_shape()[-1]

525

# Q5. Training a model

In [15]:
target = 'duration'
y_train = df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.5285191072072

In [19]:
df_feb = pd.read_parquet("../data/fhv_tripdata_2021-02.parquet")

In [20]:
df_feb['duration'] = (df_feb.dropOff_datetime - df_feb.pickup_datetime)
df_feb['duration'] = df_feb.duration.apply(lambda x: x.total_seconds()/60)
df_feb = df_feb[((df_feb.duration >= 1) & (df_feb.duration <=60))]
df_feb.PUlocationID = df_feb.PUlocationID.fillna("-1")
df_feb.DOlocationID = df_feb.DOlocationID.fillna("-1")
categorical = ['PUlocationID', 'DOlocationID']
df_feb[categorical] = df_feb[categorical].astype(str)
test_dicts = df_feb[categorical].to_dict(orient='records')
X_test = dv.transform(test_dicts)
y_test = df_feb[target].values

In [21]:
mean_squared_error(y_test, lr.predict(X_test), squared=False)

11.014283163400654